In [2]:
!pip install pandas fiona shapely pyproj rtree # à faire obligatoirement en premier pour utiliser rtree ou pygeos pour les jointures spatiales
!pip install contextily
!pip install geopandas
!pip install geoplot

     |████████████████████████████████| 15.4 MB 5.0 MB/s 
     |████████████████████████████████| 6.3 MB 39.6 MB/s 
     |████████████████████████████████| 994 kB 67.3 MB/s 
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 10.8 MB 4.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 24.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 12.1 MB 30.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 219 kB 50.3 MB/s 
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.whl size=12516304 sha256=f816ea1e684a1b6c5ef2e5f64a82f9ab77eebbdd43d6111eb6213d09d9912700


In [24]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt

import seaborn as sns

import geoplot
import folium
from IPython.core.display import display, HTML

from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

import plotly.express as px

import folium.plugins as plugins



In [5]:
df = pd.read_csv('lieux-de-tournage-a-paris-csv.csv',sep=";",engine='python')
df.head()

,Identifiant du lieu,Année du tournage,Type de tournage,Titre,Réalisateur,Producteur,Localisation de la scène,Code postal,Date de début,Date de fin,Coordonnée en X,Coordonnée en Y,geo_shape,geo_point_2d
0,2019-173,2019,Long métrage,IBRAHIM,SAMIR GUESMI,WHY NOT PRODUCTIONS,"29 boulevard auguste blanqui, 75013 paris",75013.0,2019-02-22,2019-02-22,2.352559,48.830116,"{""type"": ""Point"", ""coordinates"": [2.3525586756...","48.8301164772,2.3525586756"
1,2019-176,2019,Série TV,Le Bazar de la Charité,Alexandre Laurent,quad fantastico télévisons Le bazar de la Charité,"21 rue gracieuse, 75005 paris",75005.0,2019-03-04,2019-03-04,2.351335,48.842919,"{""type"": ""Point"", ""coordinates"": [2.3513345620...","48.8429193101,2.35133456201"
2,2019-515,2019,Série TV,PLATANE SAISON 3,ERIC JUDOR,4 MECS EN BASKETS,"9 avenue hoche, 75008 paris",75008.0,2019-05-20,2019-05-20,2.303760,48.877212,"{""type"": ""Point"", ""coordinates"": [2.3037596788...","48.8772122978,2.30375967881"
3,2019-522,2019,Série TV,The Eddy,Damien Chazelle,atlantique productions,"28 boulevard jourdan, 75014 paris",75014.0,2019-05-23,2019-05-24,2.334607,48.821424,"{""type"": ""Point"", ""coordinates"": [2.3346071504...","48.8214241259,2.33460715041"
4,2018-819,2018,Série TV,Munch saison 2,frederic Berthe,EXILENE,"rue thomas mann, 75013 paris",75013.0,2018-07-13,2018-07-13,2.378161,48.829173,"{""type"": ""Point"", ""coordinates"": [2.3781606591...","48.8291726829,2.37816065911"


In [6]:
df['Code postal'] = df['Code postal'].astype(str)
#Nombre de tournages en fonction de l'arrondissement
df['Arrondissement'] = df['Code postal'].str[3:5] #Attention aux départements qui ne commencent pas par 75
df.groupby('Arrondissement')["Code postal"].count() #Attention tournages d'un même film à plusieurs endroits
df

,Identifiant du lieu,Année du tournage,Type de tournage,Titre,Réalisateur,Producteur,Localisation de la scène,Code postal,Date de début,Date de fin,Coordonnée en X,Coordonnée en Y,geo_shape,geo_point_2d,Arrondissement
0,2019-173,2019,Long métrage,IBRAHIM,SAMIR GUESMI,WHY NOT PRODUCTIONS,"29 boulevard auguste blanqui, 75013 paris",75013.0,2019-02-22,2019-02-22,2.352559,48.830116,"{""type"": ""Point"", ""coordinates"": [2.3525586756...","48.8301164772,2.3525586756",13
1,2019-176,2019,Série TV,Le Bazar de la Charité,Alexandre Laurent,quad fantastico télévisons Le bazar de la Charité,"21 rue gracieuse, 75005 paris",75005.0,2019-03-04,2019-03-04,2.351335,48.842919,"{""type"": ""Point"", ""coordinates"": [2.3513345620...","48.8429193101,2.35133456201",05
2,2019-515,2019,Série TV,PLATANE SAISON 3,ERIC JUDOR,4 MECS EN BASKETS,"9 avenue hoche, 75008 paris",75008.0,2019-05-20,2019-05-20,2.303760,48.877212,"{""type"": ""Point"", ""coordinates"": [2.3037596788...","48.8772122978,2.30375967881",08
3,2019-522,2019,Série TV,The Eddy,Damien Chazelle,atlantique productions,"28 boulevard jourdan, 75014 paris",75014.0,2019-05-23,2019-05-24,2.334607,48.821424,"{""type"": ""Point"", ""coordinates"": [2.3346071504...","48.8214241259,2.33460715041",14
4,2018-819,2018,Série TV,Munch saison 2,frederic Berthe,EXILENE,"rue thomas mann, 75013 paris",75013.0,2018-07-13,2018-07-13,2.378161,48.829173,"{""type"": ""Point"", ""coordinates"": [2.3781606591...","48.8291726829,2.37816065911",13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8914,2020-809,2020,Long métrage,8 Rue de l'Humanité,Dany BOON,PATHÉ PROD EXÉ,"place de la bastille, 75011 paris",75011.0,2020-11-09,2020-11-11,2.369347,48.853473,"{""type"": ""Point"", ""coordinates"": [2.3693474652...","48.8534728013,2.3693474653",11
8915,2020-810,2020,Long métrage,ROBUSTE,CONSTANCE MEYER,DHARAMSALA,"10 rue vergniaud, 75013 paris",75013.0,2020-11-25,2020-11-25,2.345645,48.829516,"{""type"": ""Point"", ""coordinates"": [2.3456453654...","48.8295163483,2.34564536541",13
8916,2020-821,2020,Série TV,Arsène Lupin,Hugo GÉLIN,Gaumont production télévision,"58 quai des orfèvres, 75001 paris",75001.0,2020-11-11,2020-11-12,2.341937,48.856408,"{""type"": ""Point"", ""coordinates"": [2.3419369173...","48.8564082738,2.34193691732",01
8917,2020-835,2020,Long métrage,MES TRÈS CHERS ENFANTS,ALEXANDRA LECLÈRE,LES FILMS DU 24,"35 avenue du général sarrail, 75016 paris",75016.0,2020-11-13,2020-11-13,2.255583,48.843729,"{""type"": ""Point"", ""coordinates"": [2.2555831575...","48.8437289532,2.25558315755",16


In [7]:
#data frame du nombre de tournages par arrondissements 
df1 = pd.DataFrame({
    "Arrondissement" : list(range(-1,21)), 
    "count" : df.groupby('Arrondissement')["Code postal"].count()
})
df1

,Arrondissement,count
Arrondissement,,
,-1,1
00,0,4
01,1,497
02,2,219
03,3,164
04,4,497
05,5,469
06,6,327
07,7,472


In [8]:
#calcul du centre de la fenetre

center = df[['Coordonnée en Y', 'Coordonnée en X']].mean().values.tolist()
print(center)


[48.86261711898542, 2.3433297148929237]


In [9]:
#création de la carte et application du centre 

c = folium.Map(location=center)
c.save('maCarte1.html')

In [10]:
#définition des coins sud-ouest et nord-est de la fenetre

sw = df[['Coordonnée en Y', 'Coordonnée en X']].min().values.tolist()
ne = df[['Coordonnée en Y', 'Coordonnée en X']].max().values.tolist()
print(sw)


[48.764193, 2.22901914]


In [11]:
#application à la carte

c.fit_bounds([sw, ne])

In [12]:
#Importation des donéées geojson de Paris 

from urllib.request import urlopen
import json
with urlopen('https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=geojson&timezone=Europe/Berlin&lang=fr') as arr:
    paris = json.load(arr)

paris["features"][0]

{'geometry': {'coordinates': [[[2.365803615296344, 48.885544294580384],
    [2.364685889119478, 48.88436904340793],
    [2.364544531693668, 48.884376150666206],
    [2.364344324957409, 48.88438621708676],
    [2.363977239489263, 48.884392690720404],
    [2.3633265437514392, 48.88439733616923],
    [2.362596761119076, 48.88440254363395],
    [2.361536657916785, 48.88441010366775],
    [2.36141356067521, 48.884410981431735],
    [2.359759144865376, 48.88442274779835],
    [2.359562519576938, 48.884424139671744],
    [2.35952085624889, 48.88442442942144],
    [2.359356871546623, 48.88442559356127],
    [2.35933957202232, 48.884425716832666],
    [2.359299085562219, 48.884426003809246],
    [2.35911158966418, 48.884421545104196],
    [2.359106800862145, 48.88442129373912],
    [2.358941139000639, 48.88441258720145],
    [2.357292642630453, 48.88429203214098],
    [2.356511146781372, 48.884234867652395],
    [2.355932713128, 48.88419255389934],
    [2.355697966874508, 48.88417537905561],
  

In [13]:
#Nombre de tournages par arrondissements sur toute la période 

folium.Choropleth(geo_data = paris,
                  key_on = "feature.properties.c_ar",
                  data = df1,
                  columns = ["Arrondissement","count"],
                  fill_color = "OrRd",
                  legend_name="Nombre de tournages").add_to(c)
c

In [14]:
d = folium.Map(location=center)
d.save('maCarte1.html')
d.fit_bounds([sw, ne])

In [15]:
t = folium.Map(location=center)
t.save('maCarte1.html')
t.fit_bounds([sw, ne])

In [16]:
date_index = [2016,2017,2018,2019,2020]

hm = plugins.HeatMapWithTime(
    data=df,
    #index=date_index,
    #auto_play=True,
    radius=4,
    max_opacity=0.3
)

hm.add_to(t)
t

In [17]:
#styledict = {
 #   str(df1): data.to_dict(orient="index") for df1, data in styledata.items()
#}

In [18]:
#from folium.plugins import TimeSliderChoropleth


#m = folium.Map(location=center)

#g = TimeSliderChoropleth(
 #   paris.to_json(),
  #  styledict=styledict,
#).add_to(m)

#m

In [19]:
#nbh_locsNb_history = df.groupby(['Année du tournage','Arrondissement'])['Identifiant du lieu'].nunique().reset_index()
#nbh_locsNb_history.rename(columns={'id':'nb'}, inplace=True)
#nbh_locsNb_history.sort_values(['Arrondissement', 'Année du tournage'],
 #                              inplace=True)
#nbh_locsNb_history.reset_index(drop=True, inplace=True)

#nbh_geo_sorted_df = df.sort_values('Arrondissement').reset_index(drop=True)

In [20]:
#nbh_locsNb_history

In [25]:
#fonction pour afficher la carte sur git
def folium_deepnote_show(m):
    data = m.get_root().render()
    data_fixed_height = data.replace('width: 100%;height: 100%', 'width: 100%').replace('height: 100.0%;', 'height: 609px;', 1)
    m.fit_bounds([sw, ne])
    display(HTML(data_fixed_height))



#calcul du centre de la fenêtre
center = df[['Coordonnée en Y', 'Coordonnée en X']].mean().values.tolist()
#print(center)

#création de la carte et application du centre 
c = folium.Map(location=center)
c.save('maCarte1.html')

#définition des coins sud-ouest et nord-est de la fenetre
sw = df[['Coordonnée en Y', 'Coordonnée en X']].min().values.tolist()
ne = df[['Coordonnée en Y', 'Coordonnée en X']].max().values.tolist()
#print(sw)

#application à la carte
c.fit_bounds([sw, ne])

#Importation des données geojson de Paris 
from urllib.request import urlopen
import json
with urlopen('https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=geojson&timezone=Europe/Berlin&lang=fr') as arr:
    paris = json.load(arr)

#Nombre de tournages par arrondissements sur toute la période 
folium.Choropleth(geo_data = paris,
                  key_on = "feature.properties.c_ar",
                  data = df1,
                  columns = ["Arrondissement","count"],
                  fill_color = "OrRd",
                  legend_name="Nombre de tournages").add_to(c)
folium_deepnote_show(c)